In [2]:
import random
import itertools
import numpy
import math
import sys

In [3]:
def calcular_matriz_distancias(dataset):
    n_ciudades = len(dataset)
    distancias = numpy.zeros((n_ciudades,n_ciudades))
    for i in range(n_ciudades):
        for j in range(n_ciudades):
            if(i != j):
                distancias[i][j] = distancia_euclidea(dataset[i],dataset[j])
    return distancias

In [4]:
def distancia_euclidea(ciudad_1,ciudad_2):
    
    x1,y1 = ciudad_1[1 :]
    x2,y2 = ciudad_2[1 :]
    
    xd = x1 - x2;
    yd = y1 - y2;
    
    return round(math.sqrt( xd*xd + yd*yd ))

In [5]:
def calcular_coste(camino,distancias):
    coste = 0
    for indice in range(len(camino)-1):
        coste += distancias[camino[indice]][camino[indice+1]]
    coste += distancias[camino[-1]][camino[0]]
    return coste

In [6]:
def indice_vecinos(combinaciones_camino,len_dataset):
    # Indices nodos cercanos
    nodos_cercanos = []
    
    for vecino in combinaciones_camino:
        if (vecino[0] == 0):
            a1 = len_dataset - 1
        else:
            a1 = vecino[0] - 1

        if (vecino[0] == len_dataset - 1):
            s1 = 0
        else:
            s1 = vecino[0] + 1

        if (vecino[1] == 0):
            a2 = len_dataset - 1
        else:
            a2 = vecino[1] - 1

        if (vecino[1] == len_dataset - 1):
            s2 = 0
        else:
            s2 = vecino[1] + 1

        nodos_cercanos.append([a1, s1, a2, s2])
    return nodos_cercanos

In [1]:
def busqueda_tabu(semilla, dataset, num_vecinos):
    
    # Se establece la semilla inicial
    random.seed(semilla)
    
    # Variables extra
    len_dataset = len(dataset)
    distancias = calcular_matriz_distancias(dataset)
    solucion_actual = random.sample(range(len_dataset),len_dataset)
    coste_actual = calcular_coste(solucion_actual, distancias)
    num_iteraciones = 40 * len_dataset
    combinaciones_camino = list(itertools.combinations(list(range(0, len_dataset)), 2))
    len_combinaciones_camino = len(combinaciones_camino)
    evaluaciones = 1
    nodos_cercanos = indice_vecinos(combinaciones_camino,len_dataset)


    # Variable de salida del bucle
    seguir = True

    # Variables de la busqueda tabu
    lista_tabu = []
    lista_tabu_size = len_dataset/2
    lista_tabu_cabeza = 0
    
    matriz_de_frecuencia = numpy.zeros((len_dataset,len_dataset))
    
    mejor_solucion = solucion_actual.copy()
    mejor_coste = coste_actual
    
    iteraciones = 0
    while (seguir):

        
        indices = random.sample(range(len_combinaciones_camino), num_vecinos)
        mejor_vecino = []
        coste_mejor_vecino = sys.maxsize

        for i in indices:
            
            # Por cada vecino, estudiaremos su coste
            coste_candidato = coste_actual

            # Primero, por comodidad, generaremos los indices del camino
            a1 = nodos_cercanos[i][0]
            s1 = nodos_cercanos[i][1]
            a2 = nodos_cercanos[i][2]
            s2 = nodos_cercanos[i][3]
            i1 = combinaciones_camino[i][0]
            i2 = combinaciones_camino[i][1]

            if (i2 - 1 == i1):

                # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar


                coste_candidato -= distancias[solucion_actual[a1]][solucion_actual[i1]]
                coste_candidato -= distancias[solucion_actual[i2]][solucion_actual[s2]]

                # Despues sumamos el coste del nuevo camino

                coste_candidato += distancias[solucion_actual[a1]][solucion_actual[i2]]
                coste_candidato += distancias[solucion_actual[i1]][solucion_actual[s2]]

            elif(i1 == 0 and i2 == len_dataset-1):
                # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

                coste_candidato -= distancias[solucion_actual[a2]][solucion_actual[i2]]
                coste_candidato -= distancias[solucion_actual[i1]][solucion_actual[s1]]

                # Despues sumamos el coste del nuevo camino

                coste_candidato += distancias[solucion_actual[a2]][solucion_actual[i1]]
                coste_candidato += distancias[solucion_actual[i2]][solucion_actual[s1]]
            else:

                # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

                coste_candidato -= distancias[solucion_actual[a1]][solucion_actual[i1]]
                coste_candidato -= distancias[solucion_actual[i1]][solucion_actual[s1]]

                coste_candidato -= distancias[solucion_actual[a2]][solucion_actual[i2]]
                coste_candidato -= distancias[solucion_actual[i2]][solucion_actual[s2]]

                # Despues sumamos el coste del nuevo camino

                coste_candidato += distancias[solucion_actual[a1]][solucion_actual[i2]]
                coste_candidato += distancias[solucion_actual[i2]][solucion_actual[s1]]

                coste_candidato += distancias[solucion_actual[a2]][solucion_actual[i1]]
                coste_candidato += distancias[solucion_actual[i1]][solucion_actual[s2]]

            evaluaciones +=1

            tabu = False
            # Se comprueba si el vecino esta en la lista tabu
            if( any( combinacion == combinaciones_camino[i] for combinacion in lista_tabu ) ):
                # Prueba de aspiración 
                if coste_candidato >= mejor_coste:
                    tabu = True
            
            if(not tabu):
                if coste_candidato < coste_mejor_vecino:
                    coste_mejor_vecino = coste_candidato
                    mejor_vecino = combinaciones_camino[i]
            

        if(not (mejor_vecino == [])):
            buffer = solucion_actual[mejor_vecino[0]]
            solucion_actual[mejor_vecino[0]] = solucion_actual[mejor_vecino[1]]
            solucion_actual[mejor_vecino[1]] = buffer
            coste_actual = coste_mejor_vecino

            if coste_actual < mejor_coste:
                mejor_solucion = solucion_actual.copy()
                mejor_coste = coste_actual
            
            # Añadimos a la lista tabu el movimiento
            if(lista_tabu_cabeza < lista_tabu_size):
                
                if(len(lista_tabu) < lista_tabu_size):
                    lista_tabu.append(mejor_vecino)
                else:
                    lista_tabu[lista_tabu_cabeza] = mejor_vecino

                lista_tabu_cabeza+=1

            else:
                lista_tabu_cabeza = 1
                lista_tabu[0] = mejor_vecino

            # Actualizamos la matriz de frecuencia
            for indice in range(len_dataset):
                matriz_de_frecuencia[indice,solucion_actual[indice]] += 1


        iteraciones += 1
        # Reinicio
        if(iteraciones % (8 * len_dataset ) == 0 and iteraciones != num_iteraciones):
            
            lista_tabu = []
            lista_tabu_cabeza = 0

            if(random.uniform(0,1) < 0.5):
                lista_tabu_size += lista_tabu_size/2
            else:
                lista_tabu_size -= lista_tabu_size/2

            aleatorio = random.uniform(0,1)
            if(aleatorio <= 0.25):

                # Solución actual aleatoria
                solucion_actual = random.sample(range(len_dataset),len_dataset)
                coste_actual = calcular_coste(solucion_actual, distancias)

            elif(aleatorio <= 0.5):

                # Intensificamos
                solucion_actual = mejor_solucion.copy()
                coste_actual = mejor_coste.copy()

            else:
                
                # greedy (1º implementación)
                lista_analizar = list(range(len_dataset))
                solucion_actual = []
                for lista in matriz_de_frecuencia:
                    frecuencia = sys.maxsize
                    posicion = -1
                    for indice in lista_analizar:
                        if( lista[indice] == 0 ):
                            posicion = indice
                            break
                        else:
                            if(lista[indice] < frecuencia):
                                frecuencia = lista[indice]
                                posicion = indice
                    solucion_actual.append(posicion)
                    lista_analizar.remove(posicion)
                coste_actual = calcular_coste(solucion_actual, distancias)
                    
                            
        # Salida del bucle
        if iteraciones == num_iteraciones:
            seguir = False

    return [ dataset[ciudad] for ciudad in mejor_solucion], mejor_coste, evaluaciones